# Training example of the final model

In this notebook, we illustrate the procedure applied to select the <b>final architecture</b>. 

* This notebook does <b>NOT</b> reproduce the score on CrowdAI.
This is because here we only want to show the procedure to select the best network architecture that we used. In order to select the best architecture we have split the images of the data in <b>80/20% train/validation sets</b> and train it on the corresponding set. This notebook shows this procedure applied to our pre-selected best architecture. However, to obtain the final weights that give our CrowdAI score, we have trained the same architecture on the whole dataset. Moreover, we have also applied a post-processing to the obtained prediction.

* This model has <b>2 neurons</b> in the final layer.
The only difference between the training of a model with only one neuron at the final layer is that here we cannot rely on our implementation of the F1-score when we choose the `metrics` in `model.compile`. <br>
Hence, to perform a validation, we need to split manually the data at the very beginning and compute the F1-score in the local validation set only once the training has been finished.

* The <b>final weights</b> can be obtained by setting the validation ratio to 1 in the fourth cell of this notebook.

## Import

In [1]:
import sys
sys.path.insert(0,'../src/')
sys.path.insert(0,'../src/models/')
from training_and_prediction import *
from utilities import *

Using TensorFlow backend.


## Config:  choose the model and the root directory

In [2]:
from best_model import MODEL_CLASS
ROOT_DIR = '../Data/'

## Obtain the model class and print some infos

In [3]:
MODEL = MODEL_CLASS()
MODEL.summary()

Main model attributes:
	patch_size =  16
	input_size =  64
	pad_size =  24
	pad_rotate_size =  47
	final_layer_units =  2
	pool_size =  (2, 2)

Training parameters:
	learning_rate =  0.001
	epochs =  40
	batch_size =  128
	steps_per_epoch =  250

Other attributes:
	NameWeights =  model_best_Weights
	SubmissionName =  model_best_Submission.csv
	PredictionName =  model_best_prediction

VGG-like model with 2 neurons in the final layer.
Keras model summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2

# Training (80/20 train/validation)

In [4]:
X, Y = LoadImages(root_dir = ROOT_DIR)
ValidationRatio = 0.8
idx_tr, idx_val = get_idx_split_data(X.shape[0], ValidationRatio)

Loading 100 images
Loading 100 groundtruth images
Shape of imgs:  (100, 400, 400, 3)
Shape of gt_imgs:  (100, 400, 400)


In [5]:
X_tr, Y_tr = X[idx_tr], Y[idx_tr]
model = train(X_tr, Y_tr, MODEL)

Padding images using pad of:  47
Training the model.
Training (padded) images shape:  (80, 494, 494, 3)
Epochs: 40
Batch_size: 128
Steps per epoch: 250
No validation data.


Epoch 1/40
250/250 [==============================] - 934s 4s/step - loss: 0.5585 - acc: 0.7467
Epoch 2/40
250/250 [==============================] - 553s 2s/step - loss: 0.5111 - acc: 0.7464
Epoch 3/40
250/250 [==============================] - 567s 2s/step - loss: 0.4535 - acc: 0.7769
Epoch 4/40
250/250 [==============================] - 573s 2s/step - loss: 0.3829 - acc: 0.8241
Epoch 5/40
250/250 [==============================] - 582s 2s/step - loss: 0.3398 - acc: 0.8508
Epoch 6/40
250/250 [==============================] - 573s 2s/step - loss: 0.3118 - acc: 0.8653
Epoch 7/40
250/250 [==============================] - 581s 2s/step - loss: 0.2997 - acc: 0.8711
Epoch 8/40
250/250 [==============================] - 601s 2s/step - loss: 0.2905 - acc: 0.8752
Epoch 9/40
250/250 [==============================] - 599s

# After the training, compute F1 score on the validation

In [6]:
X_val, Y_val = X[idx_val], Y[idx_val]
ComputeLocalF1Score(X_val,Y_val,MODEL, MODEL.NameWeights)

Padding images using pad of:  24
Number of validation images:  20
Predicting... 
Done!
F1 score on validation set is: 0.7955652612128339


# We can continue with the training in this way

In [8]:
nameOldWeights = MODEL.NameWeights
nameNewWeights = MODEL.NameWeights + '_cont'
model = ContinueTrain(X_tr, Y_tr, MODEL, nameOldWeights, nameNewWeights, epochs_cont = 1)

Padding images using pad of:  47
Loading weights of the model from:  model_best_Weights
Restarting training...
Training (padded) images shape:  (80, 494, 494, 3)
Epochs: 40
Batch_size: 128
Steps per epoch: 250
No validation data.


Epoch 1/1
250/250 [==============================] - 636s 3s/step - loss: 0.1849 - acc: 0.9245
Training done, weights saved in: model_best_Weights_cont
